## 792. 匹配子序列的单词数 (11.17)

### 二分查找

In [14]:
from typing import List
from collections import defaultdict
class Solution:
    def numMatchingSubseq(self, s: str, words: List[str]) -> int:
        pos = defaultdict(list)
        for i, c in enumerate(s):
            pos[c].append(i)
        ans = len(words)
        for w in words:
            if len(w) > len(s):
                ans -= 1
                continue
            p = -1
            for c in w:
                ps = pos[c]
                j = bisect_right(ps, p)
                if j == len(ps):
                    ans -= 1
                    break
                p = ps[j]
        return ans

# 作者：LeetCode-Solution
# 链接：https://leetcode.cn/problems/number-of-matching-subsequences/solution/pi-pei-zi-xu-lie-de-dan-ci-shu-by-leetco-vki7/
# 来源：力扣（LeetCode）
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

## 1710. 卡车上的最大单元数

In [2]:
from typing import List
class Solution:
    def maximumUnits(self, boxTypes: List[List[int]], truckSize: int) -> int:
        boxTypes.sort(key = lambda x : x[1], reverse = True)
        cur, ans = truckSize, 0
        for x, y in boxTypes:
            if cur <= 0: break
            ans += cur * y if cur <= x else x * y
            cur -= x
        return ans

## 799. 香槟塔

In [116]:
class Solution:
    def champagneTower(self, poured: int, query_row: int, query_glass: int) -> float:
        if query_row == 0: return min(poured, 1)
        pre = [1]
        for _ in range(query_row):
            poured -= sum(pre)
            if poured <= 0: return 0
            cur = [1]
            for i in range(len(pre) - 1):
                cur.append(pre[i] + pre[i + 1])
            cur.append(1)
            pre = cur
        return max(poured * (pre[query_glass] / sum(pre)), 1)

In [114]:
poured = 100000009
query_row = 33
query_glass = 17

In [115]:
s = Solution()
s.champagneTower(poured, query_row, query_glass)

0

错误，没有考虑到同一排的杯子水不一定同时装满。

### 240ms

In [166]:
class Solution:
    def champagneTower(self, poured: int, query_row: int, query_glass: int) -> float:
        if query_row == 0: return min(poured, 1)
        def get_dp(x, n):
            # print(x)
            return max((dp[x] - 1) / 2, 0) if 0 <= x < n else 0
        
        dp = [poured]
        for _ in range(query_row):
            n = len(dp)
            cur = [0] * (n + 1)
            # print(cur)
            for i in range(n + 1):
                total = get_dp(i - 1, n) + get_dp(i, n)
                cur[i] = (max(total, 0))
            # print(cur)
            dp = cur
        
        return min(dp[query_glass], 1)

In [167]:
poured = 2
query_row = 1
query_glass = 1

In [168]:
s = Solution()
s.champagneTower(poured, query_row, query_glass)

0.5

### 80ms

In [176]:
class Solution:
    def champagneTower(self, poured: int, query_row: int, query_glass: int) -> float:
        row = [poured]
        for i in range(1, query_row + 1):
            nextRow = [0] * (i + 1)
            for j, volume in enumerate(row):
                if volume > 1:
                    nextRow[j] += (volume - 1) / 2
                    nextRow[j + 1] += (volume - 1) / 2
            row = nextRow
        return min(1, row[query_glass])

In [177]:
class Solution:
    def champagneTower(self, poured: int, query_row: int, query_glass: int) -> float:
        dp = [poured]
        for _ in range(query_row):
            n = len(dp)
            cur = [0] * (n + 1)
            # print(cur)
            for i in range(n):
                if dp[i] > 1:
                    cur[i] += (dp[i] - 1) / 2
                    cur[i + 1] += (dp[i] - 1) / 2
            # print(cur)
            dp = cur
        
        return min(dp[query_glass], 1)

In [178]:
poured = 2
query_row = 1
query_glass = 1

In [179]:
s = Solution()
s.champagneTower(poured, query_row, query_glass)

0.5

为啥一个外部的函数这么费时间呢？

## 1769. 移动所有球到每个盒子所需的最小操作数

In [7]:
from typing import List
class Solution:
    def minOperations(self, boxes: str) -> List[int]:
        pos = [i for i, b in enumerate(boxes) if b == "1"]
        res = []
        for i in range(len(boxes)):
            res.append(sum(abs(i - n) for n in pos))
        return res

In [10]:
boxes = "110"

In [11]:
s = Solution()
s.minOperations(boxes)

[1, 1, 3]

- 时间复杂度: O(n ** 2)
- 空间复杂度: O(n)

尝试优化

### 方法二：根据前一个盒子的操作数得到下一个盒子的操作数

> 记把所有球转移到当前下表为 $i$ 的盒子的操作数为 $operation$, 初始情况下当前盒子及其左侧有 $left_i$ 个球，右侧有 $right_i$ 个球。那么，已知这三者的情况下，把所有球转移到当前下标为 $i + 1$ 的盒子的操作数 $operation_{i + 1}$ 就可以由 $operation_i + left_i - right_i$ 快速得出，因为原来左侧的 $left_i$ 个球需要多操作一步，原来右侧的 $right_i$ 个球可以各少操作一部。计算完 $opeartion_{i + 1}$ 后，需要更新 $left_{i + 1}$ 和 $right_{i + 1}$。而初始的 $operation_0, left_0, right_0$可以通过模拟计算得到。

In [21]:
class Solution:
    def minOperations(self, boxes: str) -> List[int]:
        left, right, op = int(boxes[0]), 0, 0
        for i in range(1, len(boxes)):
            if boxes[i] == "1":
                right += 1
                op += i
        res = [op]
        for i in range(1, len(boxes)):
            op += left - right
            if boxes[i] == "1": 
                left += 1
                right -= 1
            res.append(op)
        return res

In [22]:
boxes = "001011"

In [23]:
s = Solution()
s.minOperations(boxes)

[11, 8, 5, 4, 3, 4]

- 时间复杂度：O(n)

## 895. 最大频率栈

In [26]:
from collections import defaultdict
class FreqStack:

    def __init__(self):
        self.cnt = defaultdict(int)
        self.map = defaultdict(list)
        self.max = 0

    def push(self, val: int) -> None:
        self.cnt[val] += 1
        self.map[self.cnt[val]].append(val)
        self.max = max(self.max, self.cnt[val])
        
    def pop(self) -> int:
        ans = self.map[self.max].pop()
        self.cnt[ans] -= 1
        if not self.map[self.max]: self.max -= 1
        return ans

## 1775. 通过最少操作次数使数组的和相等

贪心 + 计数排序

In [9]:
from collections import defaultdict, Counter
class Solution:
    def minOperations(self, nums1: List[int], nums2: List[int]) -> int:
        sum1, sum2 = sum(nums1), sum(nums2)
        if sum1 > sum2: return self.minOperations(nums2, nums1)
        # sum1 <= sum2
        diff = sum2 - sum1
        # cnt = Counter((6 - n) for n in nums1) + Counter((n - 1) for n in nums2)
        cnt = Counter([6 - n for n in nums1] + [n - 1 for n in nums2])
        step = 0
        for key in range(5, 0, -1):
            while cnt[key] > 0 and diff > 0:
                diff -= key
                cnt[key] -= 1
                step += 1
        
        return step if diff <= 0 else -1

## 1774. 最接近目标价格的甜品成本

针对每种base，回溯查找topping的最佳组合使得当前的值更接近target并且更小，遍历base返回全局的最优值。

In [12]:
class Solution:
    def closestCost(self, baseCosts: List[int], toppingCosts: List[int], target: int) -> int:
        ans = min(baseCosts)
        def dfs(c, idx):
            nonlocal ans
            # 当前枚举的数值已经大于目前最佳，返回
            if abs(ans - target) < c - target: return 
            # 当前枚举数值优于目前最佳
            if abs(ans - target) >= abs(c - target):
                # 严格小于，更新ans
                if abs(ans - target) > abs(c - target): ans = c
                # 相等取小的
                else: ans = min(ans, c)
            if idx == len(toppingCosts): return 
            dfs(c, idx + 1)  # 不选取当前配料
            dfs(c + toppingCosts[idx], idx + 1)  # 选取一份
            dfs(c + toppingCosts[idx] * 2, idx + 1)  # 选取两份
        
        for c in baseCosts:
            dfs(c, 0)
        return ans

## 2074. 反转偶数长度组的节点

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [28]:
from typing import Optional
class Solution:
    def reverseEvenLengthGroups(self, head: Optional[ListNode]) -> Optional[ListNode]:
        i = 0
        cur, pre = head, None
        while cur:
            i += 1
            it = cur
            length = 0
            while length < i and it:
                length += 1
                it = it.next
            
            if length & 1:
                for j in range(length):
                    pre, cur = cur, cur.next
            else:
                for j in range(length - 1):
                    pre.next, cur.next.next, cur.next = cur.next, pre.next, cur.next.next
                pre, cur = cur, cur.next

        return head

## 947. 移除最多的同行或同列石头

In [53]:
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        self.rank = [1] * n
        
    def find(self, x):
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx != fy:
            if self.rank[fx] > self.rank[fy]:
                fx, fy = fy, fx
            if self.rank[fx] == self.rank[fy]:
                self.rank[fy] += 1
            self.fa[fx] = fy

from typing import List
class Solution:
    def removeStones(self, stones: List[List[int]]) -> int:
        n = len(stones)
        # stones = [set(n) for n in stones]
        uf = UF(n)
        for i in range(n):
            for j in range(i + 1, n):
                if stones[i][0] == stones[j][0] or stones[i][1] == stones[j][1]:
                    uf.union(i, j)
        
        return n - sum(uf.fa[i] == i for i in range(n))

In [54]:
stones = [[0,0]]

In [55]:
s = Solution()
s.removeStones(stones)

0

优化：哈希表

In [59]:
from collections import defaultdict
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        self.rank = [1] * n
        
    def find(self, x):
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx != fy:
            if self.rank[fx] > self.rank[fy]:
                fx, fy = fy, fx
            if self.rank[fx] == self.rank[fy]:
                self.rank[fy] += 1
            self.fa[fx] = fy
            

class Solution:
    def removeStones(self, stones: List[List[int]]) -> int:
        cnt = defaultdict(list)
        for i, xy in enumerate(stones):
            x, y = xy[0], xy[1]
            cnt[(x, -1)].append(i)
            cnt[(y, -2)].append(i)
        
        n = i + 1
        uf = UF(n)
        for key in cnt:
            head = cnt[key][0]
            for val in cnt[key]:
                uf.union(head, val)
        
        return n - sum(uf.fa[i] == i for i in range(n))

## 337. 打家劫舍Ⅲ

In [61]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [87]:
class Solution:
    def rob(self, root: Optional[TreeNode]) -> int:
        def dp(node, i):
            if (node, i) in memo: return memo[(node, i)]
            if not node: return 0
            if not node.left and not node.right:
                return node.val if i else 0
            
            ans = node.val if i else 0
            l, r = node.left, node.right
            
            if i == 0:
                ans += max(
                    dp(l, 0) + dp(r, 0),
                    dp(l, 1) + dp(r, 1),
                    dp(l, 1) + dp(r, 0),
                    dp(l, 0) + dp(r, 1), 
                )
            else: ans += dp(l, 0) + dp(r, 0)
            
            memo[(node, i)] = ans
            return memo[(node, i)]
        
        memo = {}
        # print(memo)
        return max(dp(root, 1), dp(root, 0))

In [88]:
import modules

In [89]:
null = None
nums = root = [3,2,3,null,3,null,1]

In [90]:
head = modules.constructTree(nums)

In [91]:
s = Solution()
s.rob(head)

7

## 素数筛

In [92]:
def primeselect(right):
    s = set()
    res = []
    for i in range(2, right + 1):
        if i not in s:
            res.append(i)
            cur = i
            while cur < right:
                cur += i
                s.add(cur)
    return res

## bisect_left 和 bisect_right

In [101]:
def bisect_left(a, x):
    l, r = 0, len(a)
    while l < r:
        mid = (l + r) // 2
        if a[mid] < x: l = mid + 1
        else: r = mid
    return l

In [99]:
def bisect_right(a, x):
    l, r = 0, len(a)
    while l < r:  # 终止条件为 l 越界
        mid = (l + r) // 2
        if x < a[mid]: r = mid
        else: l = mid + 1  # 一直向右侧逼近
    return l

In [102]:
nums = [1,2,3,4,4,4,5,6]
bisect_left(nums, 4)

3

In [103]:
def bisearch(a, x):
    l, r = 0, len(a)
    while l <= r:
        mid = (l + r) // 2
        if a[mid] == x: return mid
        if a[mid] > x: r = mid - 1
        else: l = mid + 1
    return -1

In [110]:
bisearch(nums, 4)

4

## 1827. 最少操作使数组递增（12.11）

In [2]:
from typing import List
class Solution:
    def minOperations(self, nums: List[int]) -> int:
        ans = 0
        for i, n in enumerate(nums):
            if i == 0 or n > nums[i - 1]: continue
            ans += nums[i - 1] + 1
            nums[i] = nums[i - 1] + 1
        return ans

## 1781. 所有子字符串美丽值之和 (12.12)

In [1]:
from collections import defaultdict
class Solution:
    def beautySum(self, s: str) -> int:
        ans = 0
        for i in range(len(s)):
            cnt = defaultdict(int)
            mx = 0
            for j in range(i, len(s)):
                cnt[s[j]] += 1
                mx = max(mx, cnt[s[j]])
                ans += mx - min(cnt.values())
        return ans

## 1828. 统计一个圆中点的数目 (1.24)

In [6]:
from typing import List
class Solution:
    def countPoints(self, points: List[List[int]], queries: List[List[int]]) -> List[int]:
        res = [0] * len(queries)
        for i, (x1, y1, r) in enumerate(queries):
            for x2, y2 in points:
                if (x1 - x2) ** 2 + (y1 - y2) ** 2 <= r ** 2:
                    res[i] += 1
        return res 

In [7]:
points = [[1,3],[3,3],[5,3],[2,2]]
queries = [[2,3,1],[4,3,1],[1,1,2]]

In [8]:
s = Solution()
s.countPoints(points, queries)

[3, 2, 2]

## 2303. 计算应缴税款总额

In [12]:
class Solution:
    def calculateTax(self, brackets: List[List[int]], income: int) -> float:
        pre = res = 0
        for step, p in brackets:
            cur = income - pre 
            if cur <= 0: return res 
            res += min(step - pre, cur) * p / 100
            pre = step 
        return res 

In [15]:
brackets = [[2,50]]
income = 0

In [16]:
s = Solution()
s.calculateTax(brackets, income)

0

## 1663. 具有给定数值的最小字符串 (1.26)

In [14]:
class Solution:
    def getSmallestString(self, n: int, k: int) -> str:
        res = ""
        for i in range(n - 1, -1, -1):
            if (k - 26) > i: 
                res += "z"
                k -= 26 
            else:
                res += chr(96 + k - i) + "a" * i 
                break
        return res[::-1]

In [27]:
n = 5
k = 27

In [28]:
s = Solution()
s.getSmallestString(n, k)

'aaaaw'

返回字符串的都可以用list最后再join避免创建多个对象

In [26]:
class Solution:
    def getSmallestString(self, n: int, k: int) -> str:
        res = []
        for i in range(n - 1, -1, -1):
            if (k - 26) > i: 
                res.append("z")
                k -= 26 
            else:
                res.append(chr(96 + k - i)) 
                res.extend(["a"] * i)
                break
        return "".join(res)[::-1]

## 2309. 兼具大小写的最好英文字母 (1.27)

In [9]:
from collections import Counter
class Solution:
    def greatestLetter(self, s: str) -> str:
        cnt = Counter(s)
        for i in range(25, -1, -1):
            if chr(65 + i) in cnt and chr(97 + i) in cnt:
                return chr(65 + i)
        return ""

## 1664. 生态城平衡数组的方案数 (1.28)

In [29]:
from typing import List
class Solution:
    def waysToMakeFair(self, nums: List[int]) -> int:
        prefix, suffix = dict(), dict()
        odd = even = 0
        for i, n in enumerate(nums):
            prefix[i] = (odd, even)
            if i & 1: odd += n
            else: even += n 
        odd = even = 0
        for i in range(len(nums) - 1, -1, -1):
            suffix[i] = (odd, even)
            if i & 1: even += nums[i]
            else: odd += nums[i]

        ans = 0
        for key in prefix:
            o1, e1 = prefix[key]
            o2, e2 = suffix[key]
            if o1 + o2 == e1 + e2:
                ans += 1
        return ans

In [32]:
nums = [0] * 100

In [33]:
s = Solution()
s.waysToMakeFair(nums)

100

## 2315. 统计星号 (1.29)

In [1]:
class Solution:
    def countAsterisks(self, s: str) -> int:
        flag = cnt = 0
        for ch in s:
            if ch == "|": flag += 1
            if not flag & 1 and ch == "*": cnt += 1
        return cnt 

## 1669. 合并两个链表 (1.30)

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [3]:
class Solution:
    def mergeInBetween(self, list1: ListNode, a: int, b: int, list2: ListNode) -> ListNode:
        a, b, cnt = a - 1, b + 1, 0
        p = list1
        left = right = None 
        while p != None:
            if cnt == a: left = p 
            if cnt == b: 
                right = p
                break
            p = p.next
            cnt += 1
        
        p = list2
        while p.next != None:
            p = p.next
        
        left.next = list2
        p.next = right 
        return list1

## 2319. 判断矩阵是否是一个X矩阵 (1.31)

In [2]:
from typing import List
class Solution:
    def checkXMatrix(self, grid: List[List[int]]) -> bool:
        cnt0 = 0
        m = len(grid)
        for i in range(m):
            if grid[i][i] == 0 or grid[i][m - 1 - i] == 0:
                return False
            for j in range(m):
                if grid[i][j] == 0:
                    cnt0 += 1
        return cnt0 == m * (m - 2) + int(m & 1)

In [ ]:
[[2,0,0,0,1],[0,4,0,1,5],[0,0,5,0,0],[0,5,0,2,0],[4,0,0,0,2]]

In [3]:
int(True)

1

## 2325. 解密消息 (2.1)

In [8]:
class Solution:
    def decodeMessage(self, key: str, message: str) -> str:
        dic = {" ": " "}
        i = 0
        for ch in key:
            if ch in dic: continue
            dic[ch] = chr(97 + i)
            i += 1
        return "".join([dic[s] for s in message])


In [11]:
key = "eljuxhpwnyrdgtqkviszcfmabo"
message = "zwx hnfx lqantp mnoeius ycgk vcnjrdb"

In [12]:
s = Solution()
s.decodeMessage(key, message)

'the five boxing wizards jump quickly'